In [1]:
#install.packages('tseries')
#install.packages('CombMSC')
library(CombMSC)
library(tseries)
library(zoo)


spy_zoo = get.hist.quote(instrument = "SPY", "2000-01-01", "2011-12-31",quote = c("AdjClose"),
provider = c("yahoo"), method = NULL, compression = "d",
retclass = c("zoo"), quiet = FALSE, drop = FALSE)


#spy$train =ts(spy$train, frequency=7)

test_len = 504
if(FALSE){
spy = coredata(spy_zoo)
spy
spy$train = spy[1:(length(spy)-test_len)]
spy$test = spy[(length(spy)-test_len + 1):length(spy)]
length(spy$train)
length(spy$test)

plot(spy$train,ylab='price', xlab='day in train')
plot(log(spy$train), ylab="Log(price)", xlab="day in train")

plot(spy$test,ylab='price', xlab='day in test')
plot(log(spy$test), ylab="Log(price)", xlab="day in test")
}








Attaching package: ‘CombMSC’

The following object is masked from ‘package:stats’:

    BIC


Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



time series starts 2000-01-03
time series ends   2011-12-30


In [2]:
#hist(spy$train)
#hist(log(spy$train))

In [3]:
#install.packages('chron')
#require( chron )

reg_df <- read.csv(file='regression_data.csv', header = TRUE, na.strings=c("NA","NaN", "#N/A") )
reg_df = na.omit(reg_df)
#head(reg_df)

reg_df$Time = as.Date(reg_df$Time ,  format = "%m/%d/%Y")
#sapply(reg_df, class)
reg_zoo <- zoo(reg_df, order.by=reg_df$Time)

merged = merge(reg_zoo, spy_zoo, all=FALSE)


spy_df = as.data.frame(merged)



spy_df <- subset(spy_df, select = c('cpi', 'debt', 'gdp','gold','labour','party', 'election', 'petrol', 'unemploy', 'AdjClose'))

spy_df$cpi = as.numeric(as.character(spy_df$cpi))
spy_df$debt = as.numeric(as.character(spy_df$debt))
spy_df$gdp = as.numeric(as.character(spy_df$gdp))
spy_df$gold = as.numeric(as.character(spy_df$gold))
spy_df$labour = as.numeric(as.character(spy_df$labour))
spy_df$petrol = as.numeric(as.character(spy_df$petrol))
spy_df$unemploy = as.numeric(as.character(spy_df$unemploy))
spy_df$AdjClose = as.numeric(as.character(spy_df$AdjClose))


spy_train = spy_df[1:(nrow(spy_df)-test_len),]
spy_test = spy_df[(nrow(spy_df)-test_len + 1):nrow(spy_df),]
head(spy_train)


# http://ricardoscr.github.io/how-to-use-ridge-and-lasso-in-r.html


,cpi,debt,gdp,gold,labour,party,election,petrol,unemploy,AdjClose
2000-01-04,2,5628.7,10031,282.05,142267,0,1,1.312,4,101.24644
2000-01-05,2,5628.7,10031,282.10,142267,0,1,1.312,4,101.42756
2000-01-06,2,5628.7,10031,280.35,142267,0,1,1.312,4,99.79748
2000-01-07,2,5628.7,10031,282.00,142267,0,1,1.312,4,105.59334
2000-01-10,2,5628.7,10031,281.70,142267,0,1,1.304,4,105.95558
2000-01-11,2,5628.7,10031,281.70,142267,0,1,1.304,4,104.68774


In [4]:
fit.lm = lm(AdjClose ~ .,  data = spy_train)
summary(fit.lm)


Call:
lm(formula = AdjClose ~ ., data = spy_train)

Residuals:
     Min       1Q   Median       3Q      Max 
-23.5045  -3.7897   0.8021   4.7262  14.0542 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  6.994e+02  3.903e+01  17.920  < 2e-16 ***
cpi          4.301e+00  4.565e-01   9.422  < 2e-16 ***
debt         5.933e-03  4.924e-04  12.049  < 2e-16 ***
gdp          3.139e-03  9.248e-04   3.394    7e-04 ***
gold         8.306e-02  2.817e-03  29.481  < 2e-16 ***
labour      -4.729e-03  3.277e-04 -14.433  < 2e-16 ***
party1      -2.389e+01  1.105e+00 -21.623  < 2e-16 ***
election1    3.565e+00  4.376e-01   8.148 5.82e-16 ***
petrol       4.854e+00  4.999e-01   9.710  < 2e-16 ***
unemploy    -8.801e+00  2.769e-01 -31.780  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 6.739 on 2449 degrees of freedom
Multiple R-squared:  0.8247,	Adjusted R-squared:  0.824 
F-statistic:  1280 on 9 and 2449 DF,  p-v

In [5]:
lm.pred <- predict(fit.lm, spy_test)

# mse
sqrt(mean(lm.pred - spy_test$AdjClose)^2)
# rmse
sqrt(sum( (lm.pred - spy_test$AdjClose)^2 , na.rm = TRUE ) / nrow(spy_test))

[1] 29.39865

[1] 38.01323

In [15]:
install.packages("caret", dependencies = TRUE)
library('caret')

set.seed(825) # for reproducing these results

ridge <- train(AdjClose ~., data = spy_train,
               method='ridge',
               lambda = 4,
               preProcess=c('scale', 'center'))

ridge


ridge.pred <- predict(ridge, spy_test)

# mse
sqrt(mean(ridge.pred - spy_test$AdjClose)^2)
# rmse
sqrt(sum( (ridge.pred - spy_test$AdjClose)^2 , na.rm = TRUE ) / nrow(spy_test))

also installing the dependencies ‘bitops’, ‘numDeriv’, ‘gdata’, ‘caTools’, ‘TH.data’, ‘nloptr’, ‘profileModel’, ‘plotrix’, ‘KernSmooth’, ‘lava’, ‘gplots’, ‘multcomp’, ‘pbkrtest’, ‘lme4’, ‘brglm’, ‘gtools’, ‘class’, ‘plotmo’, ‘TeachingDemos’, ‘rpart’, ‘survival’, ‘prodlim’, ‘combinat’, ‘ROCR’, ‘mvtnorm’, ‘modeltools’, ‘strucchange’, ‘coin’, ‘sandwich’, ‘ISwR’, ‘corpcor’, ‘cluster’, ‘praise’, ‘car’, ‘BradleyTerry2’, ‘e1071’, ‘earth’, ‘fastICA’, ‘gam’, ‘ipred’, ‘kernlab’, ‘klaR’, ‘ellipse’, ‘mda’, ‘mlbench’, ‘MLmetrics’, ‘party’, ‘pls’, ‘pROC’, ‘proxy’, ‘randomForest’, ‘RANN’, ‘spls’, ‘subselect’, ‘pamr’, ‘superpc’, ‘Cubist’, ‘testthat’

Warning message in install.packages("caret", dependencies = TRUE):
“installation of package ‘nloptr’ had non-zero exit status”Warning message in install.packages("caret", dependencies = TRUE):
“installation of package ‘lme4’ had non-zero exit status”Warning message in install.packages("caret", dependencies = TRUE):
“installation of package ‘pbkrtest’ had 

ERROR: Error in library("caret"): there is no package called ‘caret’


In [ ]:
## k-folds
Use k-folds to select the best lambda. (Even though caret uses bootstrap in the example above by default).

For cross-validation, we will split the data into testing and training data



In [ ]:
set.seed(825)
fitControl <- trainControl(method = "cv",
                            number = 10)
# Set seq of lambda to test
lambdaGrid <- expand.grid(lambda = 10^seq(10, -2, length=100))
                          
ridge_kfold <- train(Salary ~., data = spy_train,
              method='ridge',
              trControl = fitControl,
#                tuneGrid = lambdaGrid
              preProcess=c('center', 'scale')
            )

ridge_kfold


In [ ]:
# Compute coeff
predict(ridge$finalModel, type='coef', mode='norm')$coefficients

In [ ]:
ridge_kfold.pred <- predict(ridge, spy_test)

# mse
sqrt(mean(ridge_kfold.pred - spy_test$AdjClose)^2)
# rmse
sqrt(sum( (ridge_kfold.pred - spy_test$AdjClose)^2 , na.rm = TRUE ) / nrow(spy_test))

## The lasso

In [ ]:
lasso <- train(AdjClose ~., data = spy_train,
               method='lasso',
               preProc=c('scale','center'),
              
               trControl=fitControl)
lasso

In [ ]:
# Get coef
predict.enet(lasso$finalModel, type='coefficients', s=lasso$bestTune$fraction, mode='fraction')

In [ ]:
lasso.pred <- predict(lasso, test)


# mse
sqrt(mean(lasso.pred - spy_test$AdjClose)^2)
# rmse
sqrt(sum( (lasso.pred - spy_test$AdjClose)^2 , na.rm = TRUE ) / nrow(spy_test))

In [ ]:
library(leaps)

regfit.bwd <- regsubsets(AdjClose ~ ., data=spy_train,
                      method ="backward")
summary(regfit.bwd)

In [ ]:
regfit.fwd <- regsubsets(AdjClose ~ ., data=spy_train, 
                      method = "forward")
summary(regfit.fwd)

In [ ]:


regfit <- regsubsets(AdjClose ~ ., data=spy_train)
summary(regfit)

In [ ]:
par(mfrow=c(2,2))
plot(regfit, scale = 'r2')
plot(regfit, scale = 'adjr2')
plot(regfit, scale = 'Cp')
plot(regfit, scale = 'bic')


Use the best model from leap

In [ ]:
leaplm = lm(AdjClose ~ .,  data = spy_train)
summary(leaplm)



In [ ]:
leaplm.pred <- predict(leaplm, spy_test)

# mse
sqrt(mean(leaplm.pred - spy_test$AdjClose)^2)
# rmse
sqrt(sum( (leaplm.pred - spy_test$AdjClose)^2 , na.rm = TRUE ) / nrow(spy_test))